In [1]:
# Set the Required Spark path
import findspark
findspark.init()

In [23]:
# Importing required libraries from pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_utc_timestamp, date_format

In [3]:
# creating spark session
spark = SparkSession.builder.appName("app").getOrCreate()
# creating spark context
sc = spark.sparkContext

In [4]:
# locate the csv path 
pathToRead = r"C:\Users\Sailash\Desktop\revature\Practise\game_info.csv"

In [5]:
# creating data frame from csv
raw_df = spark.read.csv(pathToRead,header=True,)

In [6]:
# printing the schema
raw_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- name: string (nullable = true)
 |-- metacritic: string (nullable = true)
 |-- released: string (nullable = true)
 |-- tba: string (nullable = true)
 |-- updated: string (nullable = true)
 |-- website: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- rating_top: string (nullable = true)
 |-- playtime: string (nullable = true)
 |-- achievements_count: string (nullable = true)
 |-- ratings_count: string (nullable = true)
 |-- suggestions_count: string (nullable = true)
 |-- game_series_count: string (nullable = true)
 |-- reviews_count: string (nullable = true)
 |-- platforms: string (nullable = true)
 |-- developers: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- publishers: string (nullable = true)
 |-- esrb_rating: string (nullable = true)
 |-- added_status_yet: string (nullable = true)
 |-- added_status_owned: string (nullable = true)
 |-- added_status_beaten: string

In [11]:
# spliting arrays from string and creating new data frame
df = raw_df.select(
    raw_df["id"],
    raw_df["slug"],
    raw_df["name"],
    raw_df["metacritic"],
    raw_df["released"],
    raw_df["tba"],
    raw_df["updated"],
    raw_df["website"],
    raw_df["rating"],
    raw_df["rating_top"],
    raw_df["playtime"],
    raw_df["achievements_count"],
    raw_df["ratings_count"],
    raw_df["suggestions_count"],
    raw_df["game_series_count"],
    raw_df["reviews_count"],
    split(col("platforms"), "\|\|").alias("platforms"),
    split(col("developers"), "\|\|").alias("developers"),
    split(col("genres"), "\|\|").alias("genres"),
    split(col("publishers"), "\|\|").alias("publishers"),
    raw_df["esrb_rating"],
    raw_df["added_status_yet"],
    raw_df["added_status_owned"],
    raw_df["added_status_beaten"],
    raw_df["added_status_toplay"],
    raw_df["added_status_dropped"],
    raw_df["added_status_playing"],
)

In [12]:
# printing the schema
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- name: string (nullable = true)
 |-- metacritic: string (nullable = true)
 |-- released: string (nullable = true)
 |-- tba: string (nullable = true)
 |-- updated: string (nullable = true)
 |-- website: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- rating_top: string (nullable = true)
 |-- playtime: string (nullable = true)
 |-- achievements_count: string (nullable = true)
 |-- ratings_count: string (nullable = true)
 |-- suggestions_count: string (nullable = true)
 |-- game_series_count: string (nullable = true)
 |-- reviews_count: string (nullable = true)
 |-- platforms: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- developers: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- publishers: array (nullable = true)
 |    |-- element: string (contai

In [57]:
# saving the
pathToSave = r"C:\Users\Sailash\Desktop\revature\Practise\game_info"
df.write.json(pathToSave)

# Which is the top most rated games for each platform

In [48]:
# selecting required columns from df
game_df = df.select(
    df["name"], 
    df["released"], 
    explode("platforms"), 
    df["rating"], 
    df["rating_top"], 
    df["ratings_count"], 
    df["playtime"]
)

In [52]:
# most rated games for each platform data frame's schema  
game_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- released: string (nullable = true)
 |-- col: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- rating_top: string (nullable = true)
 |-- ratings_count: string (nullable = true)
 |-- playtime: string (nullable = true)



In [50]:
# creating temp table
game_df.createOrReplaceTempView("filtered_games")

In [51]:
# geting the top rated game accross all platform
spark.sql("""
    select name, rating, platform from (
    select row_number() over(partition by col order by col) as num,
    name,
    col as platform,
    max(rating) over (partition by col) as rating
    from filtered_games
    where col != '0'
    order by rating desc) as table
    where num = 1
    """
).show(truncate=False)

+---------------------------------+------+----------------+
|name                             |rating|platform        |
+---------------------------------+------+----------------+
|D/Generation HD                  |5.0   |Xbox One        |
|D/Generation HD                  |5.0   |PC              |
|D/Generation HD                  |5.0   |PlayStation 4   |
|Land Sliders                     |4.86  |iOS             |
|Tron 2.0                         |4.83  |Game Boy Advance|
|Gobbo goes adventure             |4.83  |Android         |
|The Keep                         |4.78  |Nintendo 3DS    |
|Runbow                           |4.78  |Wii U           |
|D/Generation HD                  |4.78  |Nintendo Switch |
|Kane and Lynch: Dead Men         |4.78  |PlayStation 3   |
|Royal Defense Ultimate Collection|4.78  |PS Vita         |
|D/Generation HD                  |4.78  |macOS           |
|Red Entity                       |4.78  |Linux           |
|Hogwarts Legacy                  |4.73 

# Which game dev has released most games

In [17]:
# select the required columns and expand the arrays
dev_df = df.select(df["name"], explode("developers"))

In [53]:
# game developers data frame's schema
dev_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- col: string (nullable = true)



In [18]:
# creating temp table
dev_df.createOrReplaceTempView("filtered_developers")

In [44]:
# display most game released by developers
spark.sql("""
    select col as developers, 
    count(name) as games 
    from filtered_developers 
    group by col 
    order by games desc 
    limit 10
""").show(truncate=False)

+------------------------------+-----+
|developers                    |games|
+------------------------------+-----+
|Sony Interactive Entertainment|647  |
|SEGA                          |513  |
|Konami Digital Entertainment  |415  |
|Nintendo                      |395  |
|Capcom                        |395  |
|Big Fish Games                |380  |
|Electronic Arts               |316  |
|Ubisoft                       |299  |
|Square Enix                   |295  |
|Robert Brooks                 |239  |
+------------------------------+-----+



# Which game genres has most games

In [20]:
# select the required columns and expand the arrays
genres_df = df.select(df["name"], explode("genres"))

In [54]:
# game gener data frame's schema
genres_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- col: string (nullable = true)



In [21]:
# creating temp table
genres_df.createOrReplaceTempView("filtered_genres")

In [22]:
# display most game released by genres
spark.sql("""
    select col as genres, 
    count(name) as count 
    from filtered_genres 
    group by genres 
    order by count desc 
    limit 10
""").show()

+----------+------+
|    genres| count|
+----------+------+
|    Action|102023|
| Adventure| 72209|
|    Puzzle| 55550|
|Platformer| 48252|
|Simulation| 40719|
|     Indie| 33155|
|  Strategy| 33050|
|       RPG| 31947|
|   Shooter| 31822|
|    Casual| 28367|
+----------+------+



# Maximum number of games realise by years  

In [27]:
# convert date in string type into date type and create new data frame 
game_date_df = df.select(df["name"], to_utc_timestamp(date_format(col("released"), "yyy-MM-dd"), "UTC").alias("released"))

In [55]:
# max number of games realised by years data frame's schema
game_date_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- released: timestamp (nullable = true)



In [28]:
# creating temp table
game_date_df.createOrReplaceTempView("release_date")

In [43]:
# display the most number of game released per year
spark.sql("""
    select YEAR(released) as year, 
    count(name) as game_count 
    from release_date 
    group by year 
    having year is not null 
    order by game_count desc
""").show(truncate=False)

+----+----------+
|year|game_count|
+----+----------+
|2020|118507    |
|2019|79170     |
|2018|71177     |
|2017|56193     |
|2016|41074     |
|2015|26239     |
|2014|15450     |
|2013|6228      |
|2012|5268      |
|2011|4215      |
|2010|3785      |
|2009|3009      |
|2008|1940      |
|2007|1483      |
|2006|1196      |
|2004|1082      |
|2005|1078      |
|2003|1066      |
|2001|1060      |
|2002|936       |
+----+----------+
only showing top 20 rows



# DIfference of Game release and Game update 

In [31]:
# convert string type date into date type
release_update_df = df.select(
    df["name"],
    to_utc_timestamp(date_format(col("released"), "yyy-MM-dd"), "UTC").alias("released"), 
    to_utc_timestamp(date_format(col("updated"), "yyy-MM-dd"), "UTC").alias("updated"),
)

In [35]:
# convert into unix epoch and find the difference and convert into days
diff_secs_col = release_update_df["updated"].cast("long") - release_update_df["released"].cast("long")
update_df = release_update_df.withColumn("days", round(diff_secs_col / 86400, 2))

In [56]:
# game release and update difference data frame's schema
update_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- released: timestamp (nullable = true)
 |-- updated: timestamp (nullable = true)
 |-- days: double (nullable = true)



In [36]:
# creating temp table
update_df.createOrReplaceTempView("first_update")

In [42]:
# display the update difference
spark.sql("""
    select name, 
    released, 
    updated, 
    days 
    from first_update 
    where days is not null and days >= 0 
    order by days
""").show(truncate=False)

+--------------------------------------------+-------------------+-------------------+----+
|name                                        |released           |updated            |days|
+--------------------------------------------+-------------------+-------------------+----+
|PROTEST SIMULATOR                           |2019-08-28 00:00:00|2019-08-28 00:00:00|0.0 |
|l1ght                                       |2019-08-05 00:00:00|2019-08-05 00:00:00|0.0 |
|Tiny Moon Runner                            |2019-08-05 00:00:00|2019-08-05 00:00:00|0.0 |
|Steve's Indubitably Awesome Arcade Adventure|2019-08-05 00:00:00|2019-08-05 00:00:00|0.0 |
|Roll+Heart Demo                             |2019-08-05 00:00:00|2019-08-05 00:00:00|0.0 |
|vane (itch)                                 |2019-08-05 00:00:00|2019-08-05 00:00:00|0.0 |
|With your own wings                         |2019-08-05 00:00:00|2019-08-05 00:00:00|0.0 |
|Slider-Man Demo                             |2019-08-05 00:00:00|2019-08-05 00: